In [2]:
import numpy as np
import math
#用pandas读取外部文件
import pandas as pd
#绘图
import matplotlib.pyplot as plt



import numpy as np
from pandas.plotting import scatter_matrix
#sklearn分类需要的类
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

#打分
from sklearn.metrics import accuracy_score


#逻辑回归算法
from sklearn.linear_model import LogisticRegression
#交叉验证
from sklearn.model_selection import cross_val_score
#决策树
from sklearn.tree import DecisionTreeClassifier
#K-Means
from sklearn.cluster import KMeans
#贝叶斯
from sklearn.naive_bayes import GaussianNB
#支持向量机SVM
from sklearn.svm import SVC


In [4]:
test = pd.read_csv("./data/test.csv")
train = pd.read_csv("./data/train.csv")
gender_submission = pd.read_csv("./data/gender_submission.csv")
full_test = pd.merge(test, gender_submission, on="PassengerId", how="left")
data = pd.concat([full_test, train], ignore_index = True)
data

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,1
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1304,887,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,0
1305,888,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,1
1306,889,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,0
1307,890,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,1


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Pclass       1309 non-null   int64  
 2   Name         1309 non-null   object 
 3   Sex          1309 non-null   object 
 4   Age          1046 non-null   float64
 5   SibSp        1309 non-null   int64  
 6   Parch        1309 non-null   int64  
 7   Ticket       1309 non-null   object 
 8   Fare         1308 non-null   float64
 9   Cabin        295 non-null    object 
 10  Embarked     1307 non-null   object 
 11  Survived     1309 non-null   int64  
dtypes: float64(2), int64(5), object(5)
memory usage: 122.8+ KB


In [7]:
data.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Survived
count,1309.000000,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000,1309.000000
mean,655.000000,2.294882,29.881138,0.498854,0.385027,33.295479,0.377387
std,378.020061,0.837836,14.413493,1.041658,0.865560,51.758668,0.484918
min,1.000000,1.000000,0.170000,0.000000,0.000000,0.000000,0.000000
25%,328.000000,2.000000,21.000000,0.000000,0.000000,7.895800,0.000000
50%,655.000000,3.000000,28.000000,0.000000,0.000000,14.454200,0.000000
75%,982.000000,3.000000,39.000000,1.000000,0.000000,31.275000,1.000000
max,1309.000000,3.000000,80.000000,8.000000,9.000000,512.329200,1.000000


In [11]:
age_male = data.loc[data["Sex"]=="male", "Age"].dropna().mean()
age_female = data.loc[data["Sex"]=="female", "Age"].dropna().mean()
print(age_male, age_female)

30.58522796352584 28.68708762886598


In [16]:
data.loc[data["Sex"]=="male", "Age"] = data.loc[data["Sex"]=="male", "Age"].fillna(age_male)
data.loc[data["Sex"]=="female", "Age"] = data.loc[data["Sex"]=="female", "Age"].fillna(age_female)
# 即使 inplace=True 也不行

In [17]:
data["Age"].isnull().sum()

0

In [20]:
median_fare = data["Fare"].dropna().median()
data["Fare"] = data["Fare"].fillna(median_fare)
data["Fare"].isnull().sum()

0

In [22]:
data.loc[data["Cabin"].notnull(), "Cabin"] = 1
data["Cabin"] = data["Cabin"].fillna(0)
data["Cabin"].isnull().sum()

0

In [26]:
embarked_mode = data["Embarked"].dropna().mode().values[0]
data["Embarked"] = data["Embarked"].fillna(embarked_mode)
print(data["Embarked"].isnull().sum())

0


In [27]:
data.isnull().any()

PassengerId    False
Pclass         False
Name           False
Sex            False
Age            False
SibSp          False
Parch          False
Ticket         False
Fare           False
Cabin          False
Embarked       False
Survived       False
dtype: bool

In [28]:
data

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,892,3,"Kelly, Mr. James",male,34.500000,0,0,330911,7.8292,0,Q,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.000000,1,0,363272,7.0000,0,S,1
2,894,2,"Myles, Mr. Thomas Francis",male,62.000000,0,0,240276,9.6875,0,Q,0
3,895,3,"Wirz, Mr. Albert",male,27.000000,0,0,315154,8.6625,0,S,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.000000,1,1,3101298,12.2875,0,S,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1304,887,2,"Montvila, Rev. Juozas",male,27.000000,0,0,211536,13.0000,0,S,0
1305,888,1,"Graham, Miss. Margaret Edith",female,19.000000,0,0,112053,30.0000,1,S,1
1306,889,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,28.687088,1,2,W./C. 6607,23.4500,0,S,0
1307,890,1,"Behr, Mr. Karl Howell",male,26.000000,0,0,111369,30.0000,1,C,1


In [34]:
group_pclass = data.groupby("Pclass").agg({"Survived":"sum", "Fare":"mean"})
print(group_pclass)

        Survived       Fare
Pclass                     
1            186  87.508992
2            117  21.179196
3            191  13.304513


In [4]:
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [5]:
gender_submission

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
